Conexión de la DB

In [12]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de las variables de entorno
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
db = os.getenv('POSTGRES_DB')
host = os.getenv('POSTGRES_HOST', 'localhost')

# Crear la cadena de conexión utilizando las variables de entorno
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{db}')


In [13]:
# Definir la consulta SQL para obtener todos los datos de la tabla 'candidates'
query = text("SELECT * FROM candidates")

# Ejecutar la consulta y cargar los datos en un DataFrame de pandas
with engine.connect() as connection:
    df = pd.read_sql(query, connection)

# Mostrar las primeras filas del DataFrame para inspeccionar
df.head()


,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3.0,3.0
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2.0,10.0
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10.0,9.0
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7.0,1.0
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9.0,7.0


In [15]:
# Revisar la cantidad de valores nulos
df.isnull().sum()


df['seniority'] = df['seniority'].fillna('Unknown')

# Rellenar nulos en 'technical_interview_score' y 'code_challenge_score' con la media
df['technical_interview_score'] = df['technical_interview_score'].fillna(df['technical_interview_score'].mean())
df['code_challenge_score'] = df['code_challenge_score'].fillna(df['code_challenge_score'].mean())



In [16]:
# Revisar duplicados
df.duplicated().sum()

# Eliminar duplicados
df.drop_duplicates(inplace=True)


In [17]:
# Convertir 'country' a categoría para optimizar espacio si es necesario
df['country'] = df['country'].astype('category')

# Convertir 'seniority' a categoría para optimizar espacio
df['seniority'] = df['seniority'].astype('category')


In [18]:
from sqlalchemy import text

# Definir la consulta SQL para crear la nueva tabla
create_table_query = text("""
CREATE TABLE IF NOT EXISTS candidates_cleaned (
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    application_date DATE,
    country VARCHAR(50),
    yoe INTEGER,
    seniority VARCHAR(50),
    technology VARCHAR(50),
    code_challenge_score FLOAT,
    technical_interview_score FLOAT,
    year INTEGER,
    month INTEGER
);
""")

# Ejecutar la consulta para crear la tabla
with engine.connect() as connection:
    connection.execute(create_table_query)


In [19]:
# Subir los datos limpios a la nueva tabla 'candidates_cleaned'
df.to_sql('candidates_cleaned', engine, if_exists='replace', index=False)

print("Datos limpios subidos correctamente a la tabla 'candidates_cleaned'.")


Datos limpios subidos correctamente a la tabla 'candidates_cleaned'.
